In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import ee
import json
from fse_toolkit.utils.common import read_tif, statistics
from fse_toolkit.multispectral.utils import set_date_range
from fse_toolkit.multispectral.fpar import calculate_ndvi, calculate_sr, calculate_fpar
from fse_toolkit.multispectral.scraper import fetch_weather_data
from fse_toolkit.multispectral.temperature import calculate_te1, calculate_te2
from fse_toolkit.multispectral.water_stress import calculate_water_stress
from fse_toolkit.multispectral.npp import calculate_npp, calculate_apar, caculate_epsilon
from fse_toolkit.settings import EE_ACCOUNT, EE_PRIVATE_KEY_FILE

In [2]:
ee.Initialize(
    credentials=ee.ServiceAccountCredentials(
        EE_ACCOUNT,
        str(EE_PRIVATE_KEY_FILE)
    )
)

In [3]:
tif_files = [
    '../../../FPAR/To彥成/95183018_221211e_28~6584_hr4-003.tif',
    '../../../FPAR/To彥成/95183020_221120f_30~0012_hr4.tif',
    '../../../FPAR/To彥成/95183029_221120f_29~0113_hr4.tif',
    '../../../FPAR/To彥成/95183038_221211e_28~6592_hr4-019.tif',
    '../../../FPAR/To彥成/95183040_221120f_30~0004_hr4.tif',
    '../../../FPAR/To彥成/95183049_221120f_29~0121_hr4.tif'
]

In [4]:
file_id = 5
tif_data = read_tif(tif_files[file_id])
filename = Path(tif_files[file_id]).stem
print(filename)

影像的地理變換矩陣: | 0.25, 0.00, 219845.00|
| 0.00,-0.25, 2505520.00|
| 0.00, 0.00, 1.00|
影像的座標參考系統: EPSG:3826
95183049_221120f_29~0121_hr4


In [5]:
date = tif_files[1].split('_')[1][:-1]
#parse date YYMMDD -> datetime
date = datetime.strptime(date, '%y%m%d')
print("date:", date)

start_ym = date.strftime('%Y-%m')
opt_ym = datetime(date.year, 6, 1).strftime('%Y-%m') # June of the same year
print("start_ym:", start_ym, "|", "opt_ym:", opt_ym)

# 近三年八月
start_date, end_date = set_date_range(date.year, month=8, years=3)
print("start_date:", start_date, "|", "end_date:", end_date)

date: 2022-11-20 00:00:00
start_ym: 2022-11 | opt_ym: 2022-06
start_date: 2020-08-01 | end_date: 2022-08-31


In [6]:
# 檢查 output 資料夾中是否有 filename 的資料夾，若無則建立
output_dir = f'output/{filename}'
Path(output_dir).mkdir(parents=True, exist_ok=True)

### FPAR

In [7]:
ndvi = calculate_ndvi(tif_data["bands"][3], tif_data["bands"][0])
sr = calculate_sr(ndvi)
fpar = calculate_fpar(ndvi, sr, 'field')

In [8]:
ndvi_stats = statistics(ndvi)
sr_stats = statistics(sr)
fpar_stats = statistics(fpar)

In [9]:
# 把 ndvi, sr, fpar 的統計資料寫入 pandas dataframe, (ndvi, sr, fpar 為 index)
df_stats = pd.DataFrame({
    'NDVI': ndvi_stats,
    'SR': sr_stats,
    'FPAR': fpar_stats
})

In [10]:
df_stats.transpose()

,max,min,mean,std,median,count,sum
NDVI,0.970000,0.275001,0.611621,0.081445,0.619936,138641564.0,84796160.0
SR,65.666733,1.758625,4.385656,1.257206,4.262275,138641564.0,608034176.0
FPAR,0.950000,0.001000,0.460646,0.111211,0.471998,138641564.0,63864748.0


### Weather data

In [11]:
if date.month < 6:
    weather_data = fetch_weather_data(
        tif_data['coordinates']['center'][0], tif_data['coordinates']['center'][1], 
        ['平均氣溫(℃)', '累積日射量(MJ/m2)'],
        start_ym, opt_ym
    )
else:
    weather_data = fetch_weather_data(
        tif_data['coordinates']['center'][0], tif_data['coordinates']['center'][1], 
        ['平均氣溫(℃)', '累積日射量(MJ/m2)'],
        opt_ym, start_ym
    )

/Users/azetry/Documents/Projects/2024/FSES/v2/fse_toolkit/src/fse_toolkit/data/station_record.csv
WebDriver setup completed. Temp dir: /Users/azetry/Documents/Projects/2024/FSES/v2/fse_toolkit/examples/temp/tmp3rx3x77n


In [12]:
sol = weather_data['data'].loc[start_ym, '累積日射量(MJ/m2)']
t = weather_data['data'].loc[start_ym, '平均氣溫(℃)']
t_opt = weather_data['data'].loc[opt_ym, '平均氣溫(℃)']
print("sol:", sol, "|", "t:", t, "|", "t_opt:", t_opt)

sol: 554.1 | t: 25.7 | t_opt: 28.8


In [13]:
te1 = calculate_te1(t_opt)
te2 = calculate_te2(t_opt, t)
print("te1:", te1, "|", "te2:", te2)

te1: 0.9612800000000001 | te2: 0.9257429758096202


### Water stress

In [14]:
water_stress = calculate_water_stress(tif_data['coordinates'], start_date, end_date, output_numpy=True)
wstress_value = np.nanmean(water_stress)
print("water stress:", wstress_value)

water stress: 0.9663037


In [15]:
wstress_stats = statistics(water_stress)
df_stats['Water Stress'] = wstress_stats

### Epsilon

In [16]:
epsilon = caculate_epsilon(wstress_value, te1, te2)
print("epsilon:", epsilon)

epsilon: 0.8470132572526825


### APAR

In [17]:
apar = calculate_apar(fpar, sol)

In [18]:
apar_stats = statistics(apar)
df_stats['APAR'] = apar_stats

### NPP

In [19]:
npp = calculate_npp(apar, epsilon)

In [20]:
npp_stats = statistics(npp)
df_stats['NPP'] = npp_stats

In [21]:
calc_info = {
    'date': date.strftime('%Y-%m-%d'),
    'month_t_opt': opt_ym,
    'wstress_date_range': [start_date, end_date],
    'sol': sol,
    't': t,
    't_opt': t_opt,
    'te1': te1,
    'te2': te2,
    'wstress': wstress_value,
    'epsilon': epsilon
}

# to json file
calc_info_serializable = {k: float(v) if isinstance(v, np.float32) else v for k, v in calc_info.items()}
with open(f'{output_dir}/calc_info.json', 'w') as f:
    json.dump(calc_info_serializable, f)
print("calc_info saved at", f'{output_dir}/calc_info.json')

calc_info

calc_info saved at output/95183049_221120f_29~0121_hr4/calc_info.json


{'date': '2022-11-20',
 'month_t_opt': '2022-06',
 'wstress_date_range': ['2020-08-01', '2022-08-31'],
 'sol': np.float64(554.1),
 't': np.float64(25.7),
 't_opt': np.float64(28.8),
 'te1': np.float64(0.9612800000000001),
 'te2': np.float64(0.9257429758096202),
 'wstress': np.float32(0.9663037),
 'epsilon': np.float64(0.8470132572526825)}

In [22]:
df_stats.transpose().to_csv(f'{output_dir}/stats.csv')